In [140]:
import numpy as np
import pandas as pd
import re
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import preprocessing
#from wordcloud import WordCloud, STOPWORDS
#from textblob import TextBlob
import itertools

from matplotlib import pyplot as plt
import seaborn as sns

In [333]:
atl = pd.read_csv('postings_atl.csv')
bos = pd.read_csv('postings_bos.csv')
chi = pd.read_csv('postings_chi.csv')
den = pd.read_csv('postings_den.csv')
la = pd.read_csv('postings_la.csv')
nyc = pd.read_csv('postings_nyc.csv')
sc = pd.read_csv('postings_sc.csv')
sea = pd.read_csv('postings_sea.csv')
sf = pd.read_csv('postings_sf.csv')


In [334]:
atl['metro'] = list(itertools.repeat('ATL', len(atl)))
bos['metro'] = list(itertools.repeat('BOS', len(bos)))
chi['metro'] = list(itertools.repeat('CHI', len(chi)))
den['metro'] = list(itertools.repeat('DEN', len(den)))
la['metro'] = list(itertools.repeat('LA', len(la)))
nyc['metro'] = list(itertools.repeat('NYC', len(nyc)))
sc['metro'] = list(itertools.repeat('SV', len(sc)))
sea['metro'] = list(itertools.repeat('SEA', len(sea)))
sf['metro'] = list(itertools.repeat('SF', len(sf)))

In [300]:
print(list(atl.columns.values))
print(list(bos.columns.values))
print(list(chi.columns.values))
print(list(den.columns.values))
print(list(la.columns.values))
print(list(nyc.columns.values))
print(list(sc.columns.values))
print(list(sea.columns.values))
print(list(sf.columns.values))

['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title', 'metro']
['ceo_approve', 'company', 'job_descript

In [335]:
postings = pd.concat([atl, bos, chi, den, la, nyc, sc, sea, sf], axis = 0)

postings.shape

(7713, 9)

In [336]:
salary_high = []
for row in postings['salary_est']:
    try: 
        salary_high.append(row[-4:-1].strip('$'))
    except TypeError: 
        salary_high.append(np.nan)
        

In [337]:
postings['salary_low'] = postings['salary_est'].str.extract('(\d+)').astype(float)  
postings['salary_high'] = salary_high
postings['salary_high'] = postings['salary_high'].astype(float)
postings['salary_range'] = postings['salary_high'] - postings['salary_low']
postings['salary_mid'] = postings['salary_low'] + (postings['salary_range']/2)

postings.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,ceo_approve,company,job_description,location,rating,recommend_friend,salary_est,title,metro,salary_low,salary_high,salary_range,salary_mid
0,77.0,SunTrust Banks –,"Job Description, PPNR Stress Analytics Finance...","Atlanta, GA",3.1,52.0,Est. Salary $69k-$100k,PPNR Stress Analytics Finance Manager,ATL,69.0,100.0,31.0,84.5
1,86.0,Careerbuilder –,"Job Description,Data Science Fall 2017 Interns...","Norcross, GA",3.7,68.0,NaN,Data Science Fall 2017 Internship - Recommenda...,ATL,NaN,NaN,NaN,NaN
2,98.0,SPH Analytics –,"Job Description, Data Analyst ,General Duties:...","Duluth, GA",2.8,49.0,Est. Salary $58k-$77k,Data Analyst,ATL,58.0,77.0,19.0,67.5
3,96.0,T-Mobile –,"Job Description, A creative development role o...","Atlanta, GA",3.9,82.0,NaN,"Developer, Software – Workforce Management Ana...",ATL,NaN,NaN,NaN,NaN
4,67.0,LiveHealthier –,"Job Description, Position Purpose: Responsible...","Smyrna, GA",2.8,55.0,Est. Salary $53k-$68k,Data Analyst IV,ATL,53.0,68.0,15.0,60.5


In [338]:
postings['company'] = postings['company'].str.replace(' – ', '').str.strip()
postings['company'] = postings['company'].str.replace('Amazon.com', 'Amazon')
postings['company'] = postings['company'].str.replace('Amazon Web Services, Inc.', 'Amazon Web Services')
postings['company'] = postings['company'].str.replace('KPMG US', 'KPMG')

In [339]:
state = []
for row in postings['location']:
    state.append(row[-2:])

In [340]:
city = []
for row in postings['location']:
    city.append(row[:-4])

In [341]:
postings['city'] = city
postings['state'] = state


In [342]:
postings = postings.drop('location', 1)
postings = postings.drop('salary_est', 1)

In [343]:
#postings = postings[np.isfinite(postings['ceo_approve'])]  # don't want to do this
postings.index=(range(0, 7713))
postings.tail()

,ceo_approve,company,job_description,rating,recommend_friend,title,metro,salary_low,salary_high,salary_range,salary_mid,city,state
7708,36.0,Global Software Resources,"Job Description,Job Description – Data Scienti...",2.9,35.0,Data Scientist,SF,127.0,187.0,60.0,157.0,San Francisco,CA
7709,100.0,Avenue Code,"Job Description, Avenue Code is an eCommerce c...",4.3,96.0,Data Scientist,SF,116.0,160.0,44.0,138.0,San Francisco,CA
7710,100.0,Supercell,"Job Description, Description ,Food for thought...",4.4,81.0,Data Scientist,SF,97.0,135.0,38.0,116.0,San Francisco,CA
7711,88.0,The Climate Corp,"Job Description, Description ,Position Overvie...",3.9,76.0,Data Scientist,SF,138.0,190.0,52.0,164.0,San Francisco,CA
7712,-1.0,DataRobot,"Job Description,Job Summary, As a Customer Fac...",4.0,73.0,Partners - Customer Facing Data Scientist,SF,97.0,135.0,38.0,116.0,San Francisco,CA


In [197]:
#postings.to_csv('postings_master.csv', index=False)

In [344]:
# data-science indicators:

postings['python'] = postings['job_description'].str.contains('python', case=False)==True
postings['scikit'] = postings['job_description'].str.contains('scikit', case=False)==True
postings['r'] = postings['job_description'].str.contains(' r, | r |"r"', case=False)==True
postings['sql'] = postings['job_description'].str.contains('sql', case=False)==True
postings['matlab'] = postings['job_description'].str.contains('matlab', case=False)==True
postings['sas'] = postings['job_description'].str.contains(' sas', case=False)==True
postings['phd'] = postings['job_description'].str.contains('phd|ph.d|doctorate', case=False)==True
postings['masters'] = postings['job_description'].str.contains('msc|master', case=False)==True
postings['statistic'] = postings['job_description'].str.contains('statistic', case=False)==True
postings['visualization'] = postings['job_description'].str.contains('visualization', case=False)==True
postings['tableau'] = postings['job_description'].str.contains('tableau', case=False)==True
postings['ml'] = postings['job_description'].str.contains('machine learning', case=False)==True
postings['nlp'] = postings['job_description'].str.contains('natural language processing', case=False)==True
postings['hadoop'] = postings['job_description'].str.contains('hadoop', case=False)==True
postings['spark'] = postings['job_description'].str.contains('spark', case=False)==True
postings['java'] = postings['job_description'].str.contains('java', case=False)==True
postings['mongo'] = postings['job_description'].str.contains('mongo', case=False)==True
postings['hive'] = postings['job_description'].str.contains('hive', case=False)==True
postings['c#'] = postings['job_description'].str.contains('c#', case=False)==True
postings['linux'] = postings['job_description'].str.contains('linux', case=False)==True


In [345]:
# clinical scientist/medical/health care indicators:

postings['biolog'] = postings['job_description'].str.contains('biolog', case=False)==True
postings['pharm'] = postings['job_description'].str.contains('pharm', case=False)==True
postings['clinic'] = postings['job_description'].str.contains('clinic', case=False)==True
postings['immun'] = postings['job_description'].str.contains('immun', case=False)==True
postings['chemistry'] = postings['job_description'].str.contains('chemistry', case=False)==True
postings['oncology'] = postings['job_description'].str.contains('oncology', case=False)==True
postings['biochemistry'] = postings['job_description'].str.contains('biochemistry', case=False)==True
postings['neuro'] = postings['job_description'].str.contains('neuro', case=False)==True
postings['medical'] = postings['job_description'].str.contains('medical', case=False)==True
postings['disease'] = postings['job_description'].str.contains('disease', case=False)==True
postings['physician'] = postings['job_description'].str.contains('physician', case=False)==True
postings['surgeon'] = postings['job_description'].str.contains('surgeon', case=False)==True
postings['nurse'] = postings['job_description'].str.contains('nurse', case=False)==True
postings['hospital'] = postings['job_description'].str.contains('hospital', case=False)==True
postings['diseases'] = postings['job_description'].str.contains('hiv|hepatitis', case=False)==True
postings['cancer'] = postings['job_description'].str.contains('cancer', case=False)==True
postings['vaccine'] = postings['job_description'].str.contains('vaccine', case=False)==True
postings['protein'] = postings['job_description'].str.contains('protein', case=False)==True
postings['specimen'] = postings['job_description'].str.contains('specimen', case=False)==True


In [283]:
#postings.to_csv('postings_categorical.csv', index=False)


In [346]:
col_list = list(postings)


In [347]:
col_list = col_list[-19:]

col_list


['biolog',
 'pharm',
 'clinic',
 'immun',
 'chemistry',
 'oncology',
 'biochemistry',
 'neuro',
 'medical',
 'disease',
 'physician',
 'surgeon',
 'nurse',
 'hospital',
 'diseases',
 'cancer',
 'vaccine',
 'protein',
 'specimen']

In [348]:
postings_trimmed = postings

In [349]:
postings_trimmed['biol_indicator'] = postings_trimmed[col_list].sum(axis=1)

In [352]:
postings_trimmed = postings_trimmed.loc[postings_trimmed.biol_indicator<5,:]

In [353]:
col_list = list(postings_trimmed)

In [354]:
col_list = col_list[-20:]

col_list

['biolog',
 'pharm',
 'clinic',
 'immun',
 'chemistry',
 'oncology',
 'biochemistry',
 'neuro',
 'medical',
 'disease',
 'physician',
 'surgeon',
 'nurse',
 'hospital',
 'diseases',
 'cancer',
 'vaccine',
 'protein',
 'specimen',
 'biol_indicator']

In [355]:
postings_trimmed = postings_trimmed.drop(col_list, 1)

In [357]:
col_list = list(postings_trimmed)

In [358]:
col_list = col_list[-20:]

col_list

['python',
 'scikit',
 'r',
 'sql',
 'matlab',
 'sas',
 'phd',
 'masters',
 'statistic',
 'visualization',
 'tableau',
 'ml',
 'nlp',
 'hadoop',
 'spark',
 'java',
 'mongo',
 'hive',
 'c#',
 'linux']

In [359]:
postings_trimmed['data_sci_indicator'] = postings_trimmed[col_list].sum(axis=1)

In [361]:
postings_trimmed = postings_trimmed.loc[postings_trimmed.data_sci_indicator>1,:]

In [362]:
postings_trimmed = postings_trimmed.drop_duplicates()

In [363]:
postings_trimmed.index=(range(0, 4941))

In [364]:
postings_trimmed.to_csv('postings_trimmed.csv', index=False)